In [1]:
import datetime
import pandas as pd
import sys
sys.path.append('C:/Users/giaco/Git_Repositories/aCentauri')
sys.path.append('C:/Users/giaco/Git_Repositories/aCentauri/can-msg-api')

import importlib

can_db_service = importlib.import_module("can-msg-api.db.db_service")
db_service = can_db_service.DbService(out_of_folder=True)

can_db_models = importlib.import_module("can-msg-api.db.models")


Data read from C:/Users/giaco/Git_Repositories/aCentauri/can-msg-api/db/.env.


In [14]:

import datetime
import pytz
# In case the folder name is can_msg_api
# sys.path.append('C:/Users/giaco/Git_Repositories/aCentauri/can_msg_api')
# from can_msg_api.db.db_service import DbService
# from can_msg_api.db.models import BmsPackSoc, IcuHeartbeat
# db_service = DbService(out_of_folder=True)

# bms_data = db_service.query_latest(orm_model=can_db_models.BmsPackSoc, num_entries=10)
# SoC = bms_data['soc_percent']
# time_series = bms_data['timestamp']

# # Extract the single value from the Series
# timestamp_value = time_series.iloc[0]

# # Convert the value to a pandas Timestamp object
# timestamp = pd.Timestamp(timestamp_value, unit='s')  # Assuming the value is in seconds

# # Convert the Timestamp object to a Python datetime object
# datetime_obj = timestamp.to_pydatetime()

# print(datetime_obj)

# Define a timezone
tz = pytz.timezone('UTC')  # This is for UTC. Replace 'UTC' with your desired timezone string.

# Create a timezone-aware datetime object
start_time = datetime.datetime(2023, 10, 12, 12, 8, 3, tzinfo=tz)

end_time = start_time + datetime.timedelta(minutes=1)
print(start_time.timestamp())
# print(start_time.tzinfo)
bms_data = db_service.query(orm_model=can_db_models.BmsPackSoc, start_time=start_time, end_time=end_time)
SoC = bms_data['soc_percent']
print(SoC)

# icu_data = db_service.query_latest(orm_model=can_db_models.IcuHeartbeat, num_entries=10)
# # icu_data = db_service.query(orm_model=can_db_models.IcuHeartbeat, num_entries=10)
# velocity = icu_data['speed']
# print(velocity)

1697112483.0
0     0.269204
1     0.269167
2     0.269128
3     0.269091
4     0.269053
5     0.269016
6     0.268979
7     0.268940
8     0.268903
9     0.268866
10    0.268829
11    0.268792
12    0.268753
13    0.268716
14    0.268680
15    0.268643
16    0.268606
17    0.268570
18    0.268531
19    0.268494
20    0.268457
21    0.268421
22    0.268384
23    0.268345
24    0.268308
25    0.268272
26    0.268235
27    0.268198
28    0.268161
29    0.268123
30    0.268086
31    0.268050
32    0.268014
33    0.267980
34    0.267945
35    0.267911
36    0.267878
37    0.267844
38    0.267810
39    0.267774
40    0.267740
41    0.267706
42    0.267670
43    0.267644
44    0.267643
45    0.267643
46    0.267644
47    0.267645
48    0.267646
49    0.267647
50    0.267648
51    0.267650
52    0.267650
53    0.267650
54    0.267650
55    0.267650
56    0.267650
57    0.267650
58    0.267650
59    0.000000
Name: soc_percent, dtype: float64


In [4]:
import pandas as pd

# Query BmsPackSoc data
bms_data = db_service.query_latest(orm_model=can_db_models.BmsPackSoc, num_entries=10)
SoC = bms_data['soc_percent']
bms_time_series = bms_data['timestamp']

# Convert the entire time_series to a DatetimeIndex
bms_datetimes = pd.to_datetime(bms_time_series, unit='s').round('S')
# print(bms_datetimes)

# Query IcuHeartbeat data
icu_data = db_service.query_latest(orm_model=can_db_models.IcuHeartbeat, num_entries=10)
velocity = icu_data['speed']
icu_time_series = icu_data['timestamp']

# Convert the entire icu_time_series to a DatetimeIndex
icu_datetimes = pd.to_datetime(icu_time_series, unit='s')
# print(icu_datetimes)

# Calculate the mean time between the two series
mean_times = bms_datetimes + (icu_datetimes - bms_datetimes) / 2

# Round the seconds in the datetime
mean_times = mean_times.round('S')

# Create a DataFrame using the mean time as the index and the SoC and velocity values as columns
df = pd.DataFrame({
    'SoC': SoC.values,
    'velocity': velocity.values
}, index=mean_times)

df.index.name = 'time'

print(df)


                         SoC  velocity
time                                  
2023-10-12 14:02:35  0.14008  0.000000
2023-10-12 14:02:35  0.14008  0.000000
2023-10-12 14:02:34  0.14008  0.000000
2023-10-12 14:02:34  0.14008  0.000000
2023-10-12 14:02:33  0.14008  0.000000
2023-10-12 14:02:33  0.14008  0.000000
2023-10-12 14:02:32  0.14008  0.000000
2023-10-12 14:02:32  0.14008  0.415038
2023-10-12 14:02:31  0.14008  0.415038
2023-10-12 14:02:31  0.14008  0.427243


In [5]:
import requests
import constants

url = "https://api.solcast.com.au/data/forecast/radiation_and_weather?latitude=-12.456&longitude=130.845&format=json&output_parameters=air_temp,ghi,ghi10,ghi90,wind_speed_10m,wind_direction_10m&period=PT15M"

payload = {}
headers = {
  'Authorization': f'Bearer {constants.KEY_SOLCAST}'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)


{"forecasts":[{"air_temp":31,"ghi":480,"ghi10":473,"ghi90":480,"wind_speed_10m":6.2,"wind_direction_10m":298,"period_end":"2023-10-13T07:00:00.0000000Z","period":"PT15M"},{"air_temp":31,"ghi":422,"ghi10":409,"ghi90":422,"wind_speed_10m":6.2,"wind_direction_10m":292,"period_end":"2023-10-13T07:15:00.0000000Z","period":"PT15M"},{"air_temp":31,"ghi":358,"ghi10":336,"ghi90":363,"wind_speed_10m":6.1,"wind_direction_10m":287,"period_end":"2023-10-13T07:30:00.0000000Z","period":"PT15M"},{"air_temp":31,"ghi":287,"ghi10":246,"ghi90":302,"wind_speed_10m":6.1,"wind_direction_10m":282,"period_end":"2023-10-13T07:45:00.0000000Z","period":"PT15M"},{"air_temp":31,"ghi":227,"ghi10":200,"ghi90":243,"wind_speed_10m":6.1,"wind_direction_10m":277,"period_end":"2023-10-13T08:00:00.0000000Z","period":"PT15M"},{"air_temp":30,"ghi":168,"ghi10":143,"ghi90":184,"wind_speed_10m":5.9,"wind_direction_10m":274,"period_end":"2023-10-13T08:15:00.0000000Z","period":"PT15M"},{"air_temp":30,"ghi":113,"ghi10":94,"ghi90":

In [6]:
from solcast import forecast
import constants

try:
    res = forecast.radiation_and_weather(
        latitude=-12.45519,
        longitude=130.841366,
        output_parameters='air_temp,ghi,wind_speed_10m,wind_direction_10m,relative_humidity',
        format='json',
        period='PT15M',
        api_key=constants.KEY_SOLCAST
    )

except Exception as e:
    print(e)

print(res.to_pandas())

                           air_temp  ghi  wind_speed_10m  wind_direction_10m  \
period_end                                                                     
2023-10-13 07:00:00+00:00        31  480             6.3                 298   
2023-10-13 07:15:00+00:00        31  422             6.3                 292   
2023-10-13 07:30:00+00:00        31  358             6.2                 287   
2023-10-13 07:45:00+00:00        31  287             6.1                 282   
2023-10-13 08:00:00+00:00        31  227             6.1                 277   
...                             ...  ...             ...                 ...   
2023-10-15 06:00:00+00:00        32  698             6.2                 305   
2023-10-15 06:15:00+00:00        32  648             6.2                 304   
2023-10-15 06:30:00+00:00        32  595             6.1                 302   
2023-10-15 06:45:00+00:00        32  540             6.0                 299   
2023-10-15 07:00:00+00:00        32  483